In [1]:
from simulation_loop import SimulationParams, simulate_many, Buffer, simulate_system
from scipy.stats import norm, uniform, gamma, expon
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
import pandas as pd
from thinning import make_lambda_scaled, thinning_sampler
np.random.seed(42)

In [3]:
p_ = 0.55
t_ = np.linspace(0, 24, 1000)

class ArrivalDistribution:
    first_round_time = 9
    second_round_time = 15
    def __init__(self, p=0.5, offset=0):
        self.p = p
        self.first_peak = norm(loc=10 + offset, scale=1)
        self.second_peak = norm(loc=15 + offset, scale=1.5)

    def pdf(self, x):
        return self.p * self.first_peak.pdf(x) + (1 - self.p) * self.second_peak.pdf(x)

    def rvs(self, size=()):
        s1 = self.first_peak.rvs(size=size)
        s2 = self.second_peak.rvs(size=size)
        choice = uniform.rvs(size=size) < self.p
        return np.where(choice, s1, s2)

discharge_dist = ArrivalDistribution()
admission_dist = ArrivalDistribution(offset=1)

arrivals = admission_dist.rvs(size=1000)
discharges = discharge_dist.rvs(size=1000)

def service_time_dist():
    service_min_time = 5/60
    return service_min_time + expon(scale=1/60).rvs()


def transport_time_dist(distance_meters, break_down_prob=0):
    if distance_meters == 0:
        return 0
    speed_meters_per_second = 1.4
    transport_time_seconds = gamma(a=distance_meters, scale=1/speed_meters_per_second).rvs()
    time_hours = transport_time_seconds / 3600

    # Add breakdown delay with specified probability
    if break_down_prob > 0 and np.random.rand() < break_down_prob:
        time_hours += 6  # 6 hours of breakdown

    return time_hours

distances = np.array([
    [10, 50, 175, 150],
    [50, 10, 0, 0],
    [175, 0, 10, 0],
    [150, 0, 0, 10],
])

n_elevators = 3

def discharge_dist():
    t_max = 24
    n_target = 100  # Approximate expected number of events
    lambda_func, scale = make_lambda_scaled(n_target, t_max)
    t_vals = np.linspace(0, t_max, 1000)
    lambda_vals = lambda_func(t_vals)
    lambda_max = np.max(lambda_vals)
    return thinning_sampler(lambda_func, lambda_max, t_max)



# Sensitivity Analysis - Base case

# Increasing the number of bots

In [5]:
robot_range = [1, 2, 3, 5, 10]
n_iters = 100

for n_robots in robot_range:
    params = SimulationParams(
        discharge_dist,
        service_time_dist,
        transport_time_dist,
        distances,
        arrival_weights=np.ones(n_elevators) / n_elevators,
        discharge_weights=np.ones(n_elevators) / n_elevators,
        n_elevators=n_elevators,
        n_robots=n_robots,
    )

    buffers, *_ = simulate_many(params, n_iters=n_iters)

    # Compute max buffer size per run

    areas = np.percentile(buffers.max(axis=(3)), 99, axis=0) 
    print(areas)

100%|██████████| 100/100 [00:44<00:00,  2.26it/s]


[[40.01  8.01  9.01  8.01]
 [ 1.    4.    4.    4.  ]]


100%|██████████| 100/100 [00:45<00:00,  2.22it/s]


[[31.02  3.    3.    3.  ]
 [ 1.    4.01  4.    4.  ]]


100%|██████████| 100/100 [00:41<00:00,  2.43it/s]


[[41.04  2.    2.01  3.  ]
 [ 1.    4.    4.    4.  ]]


100%|██████████| 100/100 [00:43<00:00,  2.29it/s]


[[34.11  2.    2.01  3.  ]
 [ 1.    4.    4.    4.  ]]


100%|██████████| 100/100 [00:49<00:00,  2.04it/s]

[[41.01  2.    3.    2.  ]
 [ 1.    4.01  4.01  4.  ]]


# Changing $\lambda$

In [6]:
lambdas = [0.01, 0.1, 1, 10, 100]  # lambda_arrival values
scales = 1 / np.array(lambdas)
n_iters = 100  # number of simulation runs

for lam in scales:
    params = SimulationParams(
        discharge_dist,
        service_time_dist,
        transport_time_dist,
        distances,
        arrival_weights=np.ones(n_elevators) / n_elevators,
        discharge_weights=np.ones(n_elevators) / n_elevators,
        n_elevators=n_elevators,
        n_robots=2,
    )

    # Run simulations
    buffers, *_ = simulate_many(params, n_iters=n_iters, lambda_arrival=lam)

    areas = np.percentile(buffers.max(axis=(3)), 99, axis=0) 
    print(areas)

100%|██████████| 100/100 [00:45<00:00,  2.17it/s]


[[32.01  2.    3.01  3.01]
 [ 1.   38.01 39.   39.  ]]


100%|██████████| 100/100 [00:43<00:00,  2.28it/s]


[[39.06  3.    4.    4.  ]
 [ 1.   24.   23.01 23.  ]]


100%|██████████| 100/100 [00:44<00:00,  2.24it/s]


[[34.05  2.01  3.02  3.01]
 [ 1.    4.    4.    4.  ]]


100%|██████████| 100/100 [00:44<00:00,  2.26it/s]


[[34.    3.01  3.    3.  ]
 [ 1.    1.    1.    1.  ]]


100%|██████████| 100/100 [00:43<00:00,  2.32it/s]

[[35.04  2.01  3.    3.  ]
 [ 1.    1.    1.    1.  ]]


# Random robot breakdowns

In [7]:
breakdown_probs = [0, 1/100, 2/100, 5/100]

n_iters = 100  # Number of simulation runs per case


for prob in breakdown_probs:

    def transport_time_dist(distance_meters):
        if distance_meters == 0:
            return 0
        speed_mps = 1.4
        transport_time = gamma(a=distance_meters, scale=1 / speed_mps).rvs()
        time_hours = transport_time / 3600
        if np.random.rand() < prob:
            time_hours += 6  # Add 6 hours if breakdown occurs
        return time_hours

    params = SimulationParams(
        discharge_dist,
        service_time_dist,
        transport_time_dist,
        distances,
        arrival_weights=np.ones(n_elevators) / n_elevators,
        discharge_weights=np.ones(n_elevators) / n_elevators,
        n_elevators=n_elevators,
        n_robots=2,
    )

    # Run simulations
    buffers, *_ = simulate_many(params, n_iters=n_iters)

    areas = np.percentile(buffers.max(axis=(3)), 99, axis=0) 
    print(areas)

100%|██████████| 100/100 [00:48<00:00,  2.04it/s]


[[35.02  3.    4.    3.  ]
 [ 1.    4.    4.    4.  ]]


100%|██████████| 100/100 [00:46<00:00,  2.16it/s]


[[57.04 31.03 30.09 28.02]
 [35.03  4.    3.01  4.  ]]


100%|██████████| 100/100 [00:45<00:00,  2.20it/s]


[[58.06 31.   31.03 34.04]
 [51.01  3.01  3.01  3.  ]]


100%|██████████| 100/100 [00:41<00:00,  2.39it/s]

[[38.02 42.03 37.02 42.  ]
 [27.13  3.    3.    3.  ]]
